<a href="https://colab.research.google.com/github/godelstheory/glamvalid/blob/master/GLAM_Fenix_Spotchecking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [1]:
project_id = 'moz-fx-data-bq-data-science'
dataset_id = 'cdowhygelund'

metrics_table = 'moz-fx-data-shared-prod.org_mozilla_firefox.metrics'
start_date = '2020-09-24'
end_date = '2020-10-05'
min_build_id = '2015764547'

In [ ]:
probes_timing_dist = [
  'performance_page_non_blank_paint',
  'perf_awesomebar_session_suggestions',
  'performance_time_dom_complete',
  'performance_time_dom_interactive',
  'performance_time_dom_content_loaded_end',
  'performance_time_load_event_start_no_preload',
  'performance_interaction_tab_switch_composite',
  'performance_time_load_event_end_preload',
  'performance_time_load_event_start',
  'performance_page_total_content_page_load',
  'performance_time_response_start',
  'performance_interaction_keypress_present_latency',
  'performance_page_non_blank_paint',
  'performance_time_dom_content_loaded_start',
  'performance_time_load_event_end_no_preload',
  'performance_time_load_event_start_preload',
  'performance_time_load_event_end',
  'geckoview_page_load_time',
  'geckoview_page_load_progress_time'
  
]

In [20]:
probes_counters = {
    'moz-fx-data-shared-prod.org_mozilla_firefox.startup_timeline':
    [
      'startup_timeline_clock_ticks_per_second',
    ],
    'moz-fx-data-shared-prod.org_mozilla_firefox.metrics':
    [
      'metrics_top_sites_count',
      'glean_upload_pending_pings'
    ]
} 


In [3]:
probes_quantity = [
  'gfx_display_count',
  'gfx_display_primary_width'
]

In [4]:
from collections import defaultdict

from numpy.random import gamma
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
from google.cloud import bigquery

In [5]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# Definitions

## Query

Perform the histogram aggregation on BigQuery.

In [ ]:
hist_query_base = """
WITH a AS (
  SELECT
    client_info.client_id AS c_id,
    client_info.app_build AS build_id,
    {probe_hist}.values AS x
  FROM `{tbl}`
  WHERE DATE(submission_timestamp) BETWEEN '{start_date}' AND '{end_date}'
  ),
b0 AS (
  SELECT c_id, 
    build_id, 
    key, 
    sum(COALESCE(value,0)) AS value
  FROM a 
  CROSS JOIN UNNEST(x)
  WHERE build_id >= '{min_build_id}'
  GROUP BY 1,2,3 
  ),
b1 AS (
  SELECT c_id, build_id, SUM(value) AS npings 
  FROM b0 
  GROUP BY 1,2),
b  AS (
  SELECT b0.c_id, b0.build_id, key, value/npings AS p 
  FROM b1 JOIN b0 ON b0.c_id=b1.c_id AND b0.build_id=b1.build_id
),
c1 AS (
  SELECT build_id, 
    COUNT(distinct(c_id)) AS nreporting, 
    COUNT(distinct(key)) AS K,
    SUM(CAST( value as FLOAT64)) as nping
  FROM b0
  GROUP BY 1),
C AS (
  SELECT b.build_id, 
    key, 
    MAX(nreporting) AS nreporting,
    MAX(K) AS K,
    MAX(nping) AS nmeasured,
    SUM(p) AS psum
  FROM b JOIN c1 ON b.build_id = c1.build_id
  GROUP BY 1,2)
SELECT build_id, key, 
  nreporting,1/K+psum AS psum,K AS K, 
  (1/K+psum)/(nreporting+1)  AS p,
  (1/K+psum)/(nreporting+1)*nmeasured as pcounts
FROM c
"""

In [10]:
scalar_query_base = """
  SELECT
    client_info.client_id AS c_id,
    client_info.app_build AS build_id,
    SUM({probe_scalar}) AS probe_sum,
    MIN({probe_scalar}) AS probe_min,
    MAX({probe_scalar}) AS probe_max,
  FROM `{tbl}`
  WHERE DATE(submission_timestamp) BETWEEN '{start_date}' AND '{end_date}'
  AND client_info.app_build >= '{min_build_id}'
  GROUP BY 1,2
"""

## Analysis

Define method to calculate mean and quantiles from client aggregate histograms.

In [ ]:
def sample_dirichlet(values, size):
  samples = dict()
  for i in range(size):
    res = list()
    for j in range(len(values)):
      res.append(gamma(shape=values.iloc[j], scale=1, size=1)[0])
    samples[i] = res/sum(res) 
  return pd.DataFrame(samples)

def summarize_hist(df):
  return {
      # 'hist': df,
      'mean': df.mean(),
      '0.05': df.quantile(0.05),
      '0.25': df.quantile(0.25),
      '0.50': df.quantile(0.50),
      '0.75': df.quantile(0.75),
      '0.95': df.quantile(0.95),
  }

def analyze_hist(hist_df, num_bts):
  dirch_dist = sample_dirichlet(hist_df.psum, num_bts)
  keys = hist_df.key.reset_index().key
  df_sampled = dirch_dist.apply(lambda x: keys.sample(10000, weights=x, replace=True).reset_index().key, 
                             axis=0)
  df_summary = summarize_hist(df_sampled)
  # return df_summary
  res = defaultdict(dict)
  for key, value in df_summary.items():
    res[key]['est'] = value.mean()
    res[key]['lower'] = value.quantile(0.025)
    res[key]['upper'] = value.quantile(0.975)
  return pd.DataFrame.from_dict(res, orient='index')


## Plotting

In [ ]:
def plot_histogram(df, title):
  build = df.build_id.iloc[0]
  data1 = go.Scatter(
          x=df['key'],
          y=df['p'],
          mode='markers',
          marker=dict(color='black', size=0.00001, line=dict(width=0))        
      )

  # Horizontal line shape
  shapes=[dict(
          type='line',
          x0 = df['key'].iloc[i],
          y0 = 0,
          x1 = df['key'].iloc[i],
          y1 = df['p'].iloc[i],
          line = dict(
              color = 'grey',
              width = 2
          )
      ) for i in range(len(df['key']))]

  layout = go.Layout(
      shapes = shapes,
      title='{title} for build {build}'.format(title=title, build=build),
      xaxis=dict(
          title='key',
          type='log'
      ),
      yaxis=dict(
          title='p',          
      ),
  )

  fig = go.Figure(data1, layout)

  fig.show()


def plot_histogram_quantiles(df_summary, title):
  df = df_summary.stack().reset_index()

  fig = px.line(df[(df.level_2=='est') & (df.level_1 != 'mean')], x="build_id", 
                y=0, color='level_1', title=title, 
                labels = {
                    '0': 'Value',
                    'build_id': 'Build ID'
                })
  fig.show()


# Run

## Query

Query all the relevant probes

In [7]:
client = bigquery.Client(project=project_id)

In [ ]:
timings = dict()

for timing in probes_timing_dist:
  print('Querying {timing}'.format(timing=timing))
  probe = 'metrics.timing_distribution.{timing}'.format(timing=timing)
  hist_query = hist_query_base.format(probe_hist=probe,
                                      tbl=metrics_table, 
                                      start_date=start_date,
                                      end_date=end_date,
                                      min_build_id=min_build_id)
  
  timings[timing] = client.query(hist_query).to_dataframe()

Querying performance_page_non_blank_paint
Querying perf_awesomebar_session_suggestions
Querying performance_time_dom_complete
Querying performance_time_dom_interactive
Querying performance_time_dom_content_loaded_end
Querying performance_time_load_event_start_no_preload
Querying performance_interaction_tab_switch_composite
Querying performance_time_load_event_end_preload
Querying performance_time_load_event_start
Querying performance_page_total_content_page_load
Querying performance_time_response_start
Querying performance_interaction_keypress_present_latency
Querying performance_page_non_blank_paint
Querying performance_time_dom_content_loaded_start
Querying performance_time_load_event_end_no_preload
Querying performance_time_load_event_start_preload
Querying performance_time_load_event_end
Querying geckoview_page_load_time
Querying geckoview_page_load_progress_time


In [ ]:
counts = dict()

for tbl, counters in probes_counters.items():
  for count in counters:
    print('Querying {count}'.format(count=count))
    probe = 'metrics.counter.{count}'.format(count=count)
    scalar_query = scalar_query_base.format(probe_scalar=probe,
                                        tbl=tbl, 
                                        start_date=start_date,
                                        end_date=end_date,
                                        min_build_id=min_build_id)
    
    counts[count] = client.query(scalar_query).to_dataframe()

Querying startup_timeline_clock_ticks_per_second
Querying metrics_top_sites_count
Querying glean_upload_pending_pings


## Analyze

Utilize `build_id` corresponding to the most data (e.g., `max(nreporting)`) for plot comparison. Also, filter out builds with low `nreporting` for stats calculations.

In [ ]:
timings_res = defaultdict(dict)
for timing in timings: 
  print('Analyzing {timing}'.format(timing=timing)) 
  df = timings[timing].astype({'key': 'double'})
  if not df.empty:
    max_build_id = df.build_id.iloc[df.nreporting.idxmax()]
    timings_res[timing]['hist'] = df[df.build_id == max_build_id]
    timings_res[timing]['summary'] = df.groupby('build_id').apply(analyze_hist,
                                                                  num_bts=500)
  else:
    print('Missing data: {timing}'.format(timing=timing))
  

Analyzing performance_page_non_blank_paint
Analyzing perf_awesomebar_session_suggestions
Analyzing performance_time_dom_complete
Analyzing performance_time_dom_interactive
Analyzing performance_time_dom_content_loaded_end
Analyzing performance_time_load_event_start_no_preload
Missing data: performance_time_load_event_start_no_preload
Analyzing performance_interaction_tab_switch_composite
Missing data: performance_interaction_tab_switch_composite
Analyzing performance_time_load_event_end_preload
Missing data: performance_time_load_event_end_preload
Analyzing performance_time_load_event_start
Analyzing performance_page_total_content_page_load
Analyzing performance_time_response_start
Analyzing performance_interaction_keypress_present_latency
Analyzing performance_time_dom_content_loaded_start
Analyzing performance_time_load_event_end_no_preload
Missing data: performance_time_load_event_end_no_preload
Analyzing performance_time_load_event_start_preload
Missing data: performance_time_load_

# Plots

## Histograms

In [ ]:
for timing in timings_res:  
  df = timings_res[timing]['hist']
  plot_histogram(df, timing)

## Quantiles

In [ ]:
for timing in timings_res:  
  df = timings_res[timing]['summary']  
  plot_histogram_quantiles(timings_res[timing]['summary'], timing)